In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available. Check if your GPU drivers are properly installed.")


CUDA is available.


In [2]:
# !pip install datasets>=2.6.1
# !pip install git+https://github.com/huggingface/transformers
# !pip install librosa
# !pip install jiwer
# !pip install evaluate>=0.30
# #!pip install gradio
# !pip install -U accelerate
# !pip install mutagen
# !pip install srt

In [3]:
from huggingface_hub import login
# load the token from txt file
with open("HF_token.txt", "r") as f:
    HF_TOKEN = f.read().strip() # strip() removes the trailing "\n" if it exists
login(token=HF_TOKEN)


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/user_7542/.cache/huggingface/token
Login successful


In [4]:
import os
import librosa
import random
import numpy as np
import IPython.display as ipd
import pickle
import pandas as pd
from datasets import Dataset
from datasets import Audio
from transformers import WhisperProcessor
import mutagen.mp3
from tqdm import tqdm
import json
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, RoomSimulator
import srt
import re
from IPython.display import clear_output

NEWDATA = False
ADDTOKENS = True
NIKUD = False # False to remove the nikud
JUST_TEAMIM = False
BASE_CHAR = "@"
NUSACHIM =  ["ashkenazi", "maroko", "yerushalmi", "bavly"] #["ashkenazi", "maroko", "yerushalmi", "bavly"]


FASTTEST = False
BATCH_SIZE = 8

SR = 16000
RANDOM = True 
AUGMENT = False

LR = 1e-5
WARMUP_STEPS = 100
EVAL_STEPS = 500    
SAVE_STEPS = 1000
MAX_STEPS = 5000
DROPOUT = False # False or a number between 0 and 1
WEIGHT_DECAY = False # False or a number

EVALUATE_FIRST_STEP = True # if True, will evaluate the model after the first step

#base model 
BASE_MODEL_VERSIONS = ["tiny", "base", "small", "medium", "large", "large-v2", "large-v3"] # for v3 we need to change the log-mel spectrum
BASE_MODEL_VERSION = BASE_MODEL_VERSIONS[3] # num of model. 0=tiny 1=base... 6=large-v3
# BASE_MODEL_NAME = "openai/whisper-" + BASE_MODEL_VERSION
BASE_MODEL_NAME = "cantillation/Teamim-AllNusah-whisper-medium_Random-True_Mid"
# BASE_MODEL_NAME = "ivrit-ai/whisper-v2-d3-e3" best hebrew model (fine-tuned, large-v2 model)
# other hebrew model:
# "BenShermaister/whisper-medium-he"


RUN_NAME = BASE_MODEL_VERSION + "_Random-" + str(RANDOM) + (("_DropOut-" + str(DROPOUT)) if DROPOUT else "") + (("_WeightDecay-" + str(WEIGHT_DECAY)) if WEIGHT_DECAY else "")  + "_Augmented"*AUGMENT \
                                                                                + ("" if NEWDATA else "_OldData")   # + "_Warmup_steps-" + str(WARMUP_STEPS) + "_Eval_steps-" + str(EVAL_STEPS) + "_Save_steps-" + str(SAVE_STEPS) + "_Max_steps-" + str(MAX_STEPS) # + "_EvalFirstStep-" + str(EVALUATE_FIRST_STEP)  "_LR-" + str(LR) + 



#the new model - after training 
MODEL_NAME = f"./Teamim-{RUN_NAME}" # because the run name doesn't work, I added it to the model name
MODEL_NAME


'./Teamim-medium_Random-True_OriginalData'

In [5]:
processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="hebrew", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
if ADDTOKENS:
    teamim = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', 'ֽ']
    if JUST_TEAMIM:
        new_tokens = [BASE_CHAR + c for c in teamim] # add the base char to the teamim (e.g. א֑)
    elif NIKUD:
        new_tokens = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
    else:
        new_tokens = teamim

    processor.tokenizer.add_tokens(new_tokens)
    
    
        
        
    

In [7]:
def remove_nikud(text): #TODO IMPORT FROM THE FILE 
    nikud_list = ["ֱ","ֲ","ֳ","ִ","ֵ","ֶ","ַ","ָ","ׂ","ׁ","ֹ","ּ","ֻ","ְ","ׇ"]
    for nikud in nikud_list:
        text = text.replace(nikud, "")
    return text

def just_teamim(text, base_char = BASE_CHAR):#TODO IMPORT FROM THE FILE 
    teamim = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', 'ֽ']
    new_text = ""
    for char in text:
        if char in teamim:
            new_text += base_char
            new_text += char
        elif char == " ":
            new_text += " "
    return new_text

In [8]:
# path = "/content/drive/Othercomputers/My Laptop/Project/data/PocketTorah/"
# path = "/content/drive/MyDrive/PocketTorah/"
path = "."

In [9]:
class parashat_hashavua_dataset:
        def __init__(self, new_data, few_data=False, train = True ,validation=False, test=False, num_of_words_in_sample = 15, random = False, prob_for_num_of_parts=[], nusachim=["ashkenazi"], augment=False, load_cantillationless_data = False):
                self.data = []
                self.few_data = few_data
                self.load_data(new_data, train, validation, test, nusachim=nusachim, load_cantillationless_data=load_cantillationless_data)
                if JUST_TEAMIM:
                        self.data['text'] = self.data['text'].apply(just_teamim)
                elif not NIKUD:
                        self.data['text'] = self.data['text'].apply(remove_nikud)
                self.data = self.data[self.data['text'] != ""] # remove empty texts (and their audio)
                self.num_of_words_in_sample = num_of_words_in_sample
                self.random = random
                self.start = 0
                self.is_eval_set = validation or test
                self.prob_for_num_of_parts = prob_for_num_of_parts if prob_for_num_of_parts else [1/self.num_of_words_in_sample for i in range(self.num_of_words_in_sample)]
                self.augment = augment
                # prob_for_num_of_parts - the probability to take 1, 2, 3, etc. parts.
                # example of prob_for_num_of_parts: [0.1, 0.2, 0.3, 0.4] means that the probability to take 1 part is 0.1, 2 parts is 0.2, etc.

        def __getitem__(self, index):
                if self.is_eval_set:
                        audio, text_tokens, _ = self.get_sequence_(index*self.num_of_words_in_sample, num_of_words=self.num_of_words_in_sample)
                else:
                        if self.random:
                                # ensure that the sum of probabilities is 1
                                if np.sum(self.prob_for_num_of_parts) != 1:
                                        self.prob_for_num_of_parts = self.prob_for_num_of_parts / np.sum(self.prob_for_num_of_parts)
                                # get the number of parts
                                num_of_parts = np.random.choice(np.arange(1, len(self.prob_for_num_of_parts)+1), p=self.prob_for_num_of_parts)
                        # get the sequence
                                audio, text_tokens = self.get_random_words_sequence_audio_tokens(num_of_words=self.num_of_words_in_sample, num_of_parts=num_of_parts)
                        else:
                                audio, text_tokens, _ = self.get_sequence_(index, num_of_words=self.num_of_words_in_sample)
                if self.augment:
                        # augment the audio
                        audio = self.augment_audio(audio)

                # compute log-Mel input features from input audio array
                input_features = processor.feature_extractor(audio, sampling_rate=SR).input_features[0]
                # compute input length of audio sample in seconds
                input_length = len(audio) / SR
                # processor.tokenizer.decode(text_tokens)
                return {"input_features": input_features, "input_length": input_length, "labels": text_tokens}

        def __len__(self):
                if self.is_eval_set:
                        return int(len(self.data)/self.num_of_words_in_sample)
                else:
                        if self.random:
                                # high number because of the augmentation
                                return 100000
                        else:
                                # The length is the (number of word in the data)/(number of words in sequance)
                                return len(self.data)

        def get_sequence_audio_text(self, sequence):
                audio = np.concatenate(sequence['audio'].values)
                text = " ".join(sequence['text'])
                audio_len = len(audio) / 16000
                text_tokens = processor.tokenizer.encode(text)
                text_len = len(text_tokens)
                return sequence, audio, text, audio_len, text_tokens, text_len
        
        def load_data(self,new_data , train, validation, test, nusachim=["ashkenazi"], load_cantillationless_data = False): 
                if load_cantillationless_data:
                        self.load_data_srt_mp3(train, validation, test)
                elif new_data:
                        if  (train==True and validation==False and test==False):
                                self.load_data_new(nusachim,train=True, validation=False, test=False)
                        elif (train==False and validation==True and test==False):
                                self.load_data_new(nusachim,train=False, validation=True, test=False)
                        elif (train==False and validation==False and test==True):
                                self.load_data_new(nusachim,train=False, validation=False, test=True)
                        else:
                                print(f"Invalid input. Please provide a valid input. train={train}, validation={validation}, test={test}")
                else:
                        self.load_data_old(validation)

        # methods for the new data
        def is_mp3_and_legal_length(self, filename, min_length=0.2, max_length=20):
                try:
                        audio = mutagen.mp3.MP3(filename)
                        if audio.info.length < min_length or audio.info.length > max_length:
                                return False
                        else:
                                return True
                except mutagen.MutagenError:
                        return False

        
        def is_text_with_nikud(self, text):
                for char in text:
                        if char in "ְֱֲֳִֵֶַָֹֺֻּֽ֑֖֛֢֣֤֥֦֧֪֚֭֮֒֓֔֕֗֘֙֜֝֞֟֠֡֨֩֫֬֯־ֿ׀ׁׂ׃ׅׄ׆ׇ": # string of all the nikud characters ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
                                return True
                return False

        def is_text_and_audio_pair_legal(self, text, filename):
                if not self.is_text_with_nikud(text):
                        return False
                if not self.is_mp3_and_legal_length(filename):
                        return False
                return True

        def load_data_new(self, nusachim, train, validation, test):
                # Load dataset.json
                if train:
                        file_path = 'train_data.json'
                elif validation:
                        file_path = 'validation_data.json'
                elif test:
                        file_path = 'test_data.json'
                else:
                        file_path = '03_dataset.json'

                with open(file_path, 'r', encoding='utf-8') as f:
                        predataset = json.load(f)
                
                audios = []
                texts = []
                for nusach in nusachim:
                        file_path = "dataset_" + nusach + ".npy"
                        if os.path.exists(file_path) and False: # we don't want to use the saved data as 1 file right now
                                data = np.load(file_path, allow_pickle=True).item()
                                audios.extend(data['audio'])
                                texts.extend(data['text'])
                        else:
                                if self.few_data:
                                        predataset[nusach] = predataset[nusach][:500]
                                        predataset['text'] = predataset['text'][:500]
                                        
                                missing_files = []
                                for index, audio_file in enumerate(tqdm(predataset[nusach], desc=f"Loading {nusach} nusach ({nusachim.index(nusach)+1}/{len(nusachim)})")):
                                        audio_path = os.path.join(audio_file)
                                        if self.is_text_and_audio_pair_legal(predataset['text'][index], audio_path):
                                                audio, sr = librosa.load(audio_path, sr=SR)
                                                audios.append(audio)
                                                texts.append(predataset['text'][index])
                                        else:
                                                missing_files.append((audio_path, predataset['text'][index], index))
                                # Save the missing files
                                with open('missing_files' + nusach + '.json', 'w', encoding='utf-8') as f:
                                        json.dump(missing_files, f, ensure_ascii=False, indent=4)
                                print("Num of missing files in " + nusach + " nusach: ", len(missing_files))
                                # Save the data for the next time
                                data = {"audio": audios, "text": texts}
                # create the dataset
                self.data = {"audio": audios, "text": texts}
                self.data = pd.DataFrame(self.data)
                

        # methods for the old data
        def prepare_transcript_str_to_list_old_data(self, text:str) -> list:
                """
                this function get a string of words and return a list of the words
                """
                text = text.replace(" ׀ ", "׀").replace(" ׀ ", "׀").replace("׀", "׀ ").replace("־", "־ ").replace("[1]", "")
                text = re.sub(r'\s+|\n', ' ', text)  # replace multiple spaces or newline with a single space
                text_list = text.split(" ")
                if text_list[-1] == "":
                        text_list = text_list[:-1]
                return text_list

        def load_data_old(self, validation):
                success_count = 0
                fail_count = 0
                diff_list = []
                if validation:
                        transcript_folder = path + '/text_val'
                else:
                        transcript_folder = path + '/text'
                audio_folder = path + '/audio'
                timing_folder = path + '/time'
                audios = []
                text = []
                for filename in tqdm(os.listdir(transcript_folder)):
                        if filename.endswith(".txt"):
                                audio_path = os.path.join(audio_folder, filename.replace('.txt', '.mp3'))
                                transcript_path = os.path.join(transcript_folder, filename)
                                timing_path = os.path.join(timing_folder, filename)
                                audio, sr = librosa.load(audio_path, sr=16000)
                                with open(transcript_path, 'r', encoding='utf-8') as f:
                                        transcript = self.prepare_transcript_str_to_list_old_data(f.read())
                                with open(timing_path, 'r', encoding='utf-8') as f:
                                        timings = [float(time) for time in f.read().split(",")]
                                
                                if len(transcript) != len(timings):
                                        diff_list.append((len(transcript) - len(timings), filename))
                                        fail_count += 1
                                else:
                                        success_count += 1
                                        for i, (word, start_time) in enumerate(zip(transcript, timings)):
                                                if i == len(transcript) - 1:
                                                        end_time = len(audio) / sr
                                                else:
                                                        end_time = timings[i+1]
                                                word_audio = audio[int(start_time * sr):int(end_time * sr)]
                                                audios.append(word_audio)
                                                text.append(word)
                print("success_count: ", success_count)
                print("fail_count: ", fail_count)
                
                # diff_histogram:{1: 95, -1: 47, -6: 2, -2: 8, -5: 3, -3: 6, -7: 2, -4: 3, -8: 1, 3: 1, -9: 1, -26: 1}
                diff_histogram = {}
                for diff, filename in diff_list:
                        if diff in diff_histogram:
                                diff_histogram[diff] += 1
                        else:
                                diff_histogram[diff] = 1
                
                for key in sorted(diff_histogram.keys()):
                        print(f"{key}: {diff_histogram[key]}")
                
                print("diff_list: ", sorted(diff_list, key=lambda x: x[0]))
                self.failed_files = diff_list
                
                data_dict = {"audio": audios, "text": text}
                self.data = pd.DataFrame(data_dict)

        def check_failed_files_of_old_data(self):
                """
                check the failed files of the old data.
                listen to the audio and check the text.
                """
                for diff, filename in self.failed_files:
                        clear_output()
                        print("filename: ", filename)
                        audio_path = os.path.join(path + '/audio', filename.replace('.txt', '.mp3'))
                        transcript_path = os.path.join(path + '/text', filename)
                        timing_path = os.path.join(path + '/time', filename)
                        audio, sr = librosa.load(audio_path, sr=16000)
                        with open(transcript_path, 'r', encoding='utf-8') as f:
                                transcript = self.prepare_transcript_str_to_list_old_data(f.read())
                        with open(timing_path, 'r', encoding='utf-8') as f:
                                timings = [float(time) for time in f.read().split(",")]
                        print("num of words: ", len(transcript))
                        print("num of timings: ", len(timings))
                        
                        num = int(input("Enter the number of the word you want to listen to: "))
                        while num != -1:
                                if num == -2:
                                        # get the range that we want to search on
                                        start = int(input("Enter the start of the range: "))
                                        end = int(input("Enter the end of the range: "))
                                        num_steps = int(input("Enter the number of steps: "))
                                        for i in range(start, end, num_steps):
                                                self.check_failed_files_of_old_data_helper(timings, transcript, audio, sr, i)
                                else:
                                        self.check_failed_files_of_old_data_helper(timings, transcript, audio, sr, num)
                                num = int(input("Enter the number of the word you want to listen to: "))
                
                
        def check_failed_files_of_old_data_helper(self, timings, transcript, audio, sr, num):
                print("num: ", num)
                start_time = timings[num]
                end_time = timings[num+1] if num != len(timings) - 1 else len(audio) / sr
                print("start_time: ", start_time, " end_time: ", end_time)
                
                # print 3 words, 1 before the word, the word and 1 after the word (if exists)
                if num == 0:
                        print("word: ", transcript[num], " ", transcript[num+1])
                elif num == len(transcript) - 1:
                        print(transcript[num-1], " ", transcript[num])
                else:
                        print(transcript[num-1], " ", transcript[num], " ", transcript[num+1])
        
                # play the audio
                ipd.display(ipd.Audio(audio[int(start_time * sr):int(end_time * sr)], rate=SR))

        
        
        # For the new data without the cantillation
        def load_data_srt_mp3(self, train, validation, test):
                if train and not validation and not test:
                        folder = './train_cantillationless/'
                elif validation and not train and not test:
                        folder = './validation_cantillationless/'
                elif test and not train and not validation:
                        folder = './test_cantillationless/'
                else:
                        print("Invalid input. Please provide a valid input.")
                
                audio_folder = folder + 'audio'
                transcript_folder = folder + 'text' # the text is in srt format with times
                audios = []
                text = []
                import srt
                for filename in tqdm(os.listdir(transcript_folder)):
                        if filename.endswith(".srt"):
                                audio_path = os.path.join(audio_folder, filename.replace('.srt', '.mp3'))
                                transcript_path = os.path.join(transcript_folder, filename)
                                with open(transcript_path, 'r', encoding='utf-8') as f:
                                        transcript = list(srt.parse(f.read()))
                                audio, sr = librosa.load(audio_path, sr=16000)
                                for i, sub in enumerate(transcript):
                                        start_time = sub.start.total_seconds()
                                        end_time = sub.end.total_seconds()
                                        word_audio = audio[int(start_time * sr):int(end_time * sr)]
                                        audios.append(word_audio)
                                        text.append(sub.content)
                data_dict = {"audio": audios, "text": text}
                self.data = pd.DataFrame(data_dict)
                

                        
        def get_data(self):
                return self.data

        def get_random_word(self):
                return random.choice(self.data)

        def get_sequence(self, start, end):
                return self.data[start:end]

        # the limit of whisper model
        # audio length of 30 seconds
        # text length of 448 tokens
        # I will take 20 words and check if the audio and text are in the limit
        def get_sequence_(self, start, num_of_words=20, random_cut_long=False):
                if start + num_of_words > len(self.data):
                        end = len(self.data)
                else:
                        end = start + num_of_words
                sequence = self.get_sequence(start, end)
                sequence, audio, text, audio_len, text_tokens, text_len = self.get_sequence_audio_text(sequence)
                if audio_len < 30 and text_len < 448:
                        return audio, text_tokens, end
                else: # cut the sequence
                        print("this sequence of ", num_of_words, " words is too long!")
                        print("sequence audio length: ", audio_len)
                        print("sequence text length(in tokens): ", text_len)
                        print("text: ", text)
                        # ipd.display(ipd.Audio(audio, rate=SR))

                        if random_cut_long:
                                # divide into 2 parts and randomaly take one of them
                                if random.randint(0, 1) == 0:
                                        start = start + int(num_of_words/2)

                        if num_of_words>=2:
                                return self.get_sequence_(start, num_of_words=int(num_of_words/2), random_cut_long=random_cut_long)
                        else:
                                return self.get_sequence_(end, num_of_words=num_of_words, random_cut_long=random_cut_long)


        def get_dataset_slice_to_sequences(self, num_of_words):
                audios = []
                labels = []
                start = 0
                while start < len(self.data):
                        audio, label_feature, start = self.get_sequence_(start,num_of_words)
                        audios.append(audio)
                        labels.append(label_feature)
                dataset = {"audios": audios, "labels": labels}
                dataset = pd.DataFrame(dataset)
                return dataset
        

        def get_random_sequence_(self, length=20):
                """
                get random sequence of "length" words
                """
                start = random.randint(0, len(self.data) - length)
                return self.get_sequence_(start)

        def get_random_sequence(self, length=20):
                """
                get random sequence of "length" words
                """
                start = random.randint(0, len(self.data) - length)
                return self.get_sequence(start, start+length)

        def get_random_words_sequence_audio_tokens(self, num_of_words, num_of_parts = None):
                """
                get sequence of random words (not logical sentences)
                createed from num_of_parts short sentences
                """
                if num_of_parts == None:
                        num_of_parts = num_of_words

                if num_of_parts > num_of_words:
                        print("num_of_parts can't be bigger than num_of_words")
                        print("so num_of_parts = num_of_words = ", num_of_words)
                        num_of_parts = num_of_words

                # num of words in each part
                num_of_words_in_parts = [num_of_words // num_of_parts + (1 if i < num_of_words % num_of_parts else 0) for i in range(num_of_parts)]

                sequence = {"audio": [], "text": []}
                for num_of_words_in_part in num_of_words_in_parts:
                        part = self.get_random_sequence(num_of_words_in_part)
                        sequence["audio"].extend(part["audio"])
                        sequence["text"].extend(part["text"])
                sequence = pd.DataFrame(sequence)
                sequence, audio, text, audio_len, text_tokens, text_len = self.get_sequence_audio_text(sequence)
                if audio_len < 30 and text_len < 448:
                        return audio, text_tokens
                else:
                        print("this sequence (of ", num_of_words, " words) is too long!")
                        print("sequence audio length: ", audio_len)
                        print("sequence text length(in tokens): ", text_len)
                        print("text: ", text)
                        # ipd.display(ipd.Audio(audio, rate=SR))
                        return self.get_random_words_sequence_audio_tokens(num_of_words, num_of_parts)


        def get_dataset_slice_to_sequences_random_words(self, num_of_words, num_of_sequences=None, times = 5):
                audios = []
                labels = []
                if num_of_sequences:
                        num_of_sequences = num_of_sequences
                else:
                        num_of_sequences = int(len(self.data)*times/num_of_words)
                for i in range(num_of_sequences):
                        audio, label_feature = self.get_random_words_sequence_audio_tokens(num_of_words)
                        audios.append(audio)
                        labels.append(label_feature)
                dataset = {"audios": audios, "labels": labels}
                dataset = pd.DataFrame(dataset)
                return dataset
        
        
        # methods for checking the data
        def get_longest_audio_index(self):
                """
                returns the index of longest audio in the dataset
                """
                index = np.argmax([len(audio) for audio in self.data['audio']])
                return index
        
        def get_longest_text_index(self):
                """
                returns the index of longest text in the dataset
                """
                index = np.argmax([len(text) for text in self.data['text']])
                return index
        
        def get_shortest_audio_index(self):
                """
                returns the index of shortest audio in the dataset
                """
                index = np.argmin([len(audio) for audio in self.data['audio']])
                return index
        
        def get_shortest_text_index(self):
                """
                returns the index of shortest text in the dataset
                """
                index = np.argmin([len(text) for text in self.data['text']])
                return index
        
        def check_the_data(self):
                """
                find the longest and shortest audio and text in the dataset
                and print and play them
                """
                index = self.get_longest_audio_index()
                print("longest audio index: ", index)
                print("longest audio text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_shortest_audio_index()
                print("shortest audio index: ", index)
                print("shortest audio text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_longest_text_index()
                print("longest text index: ", index)
                print("longest text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

                index = self.get_shortest_text_index()
                print("shortest text index: ", index)
                print("shortest text: ", self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))


        def remove_word_by_index(self, index):
                """
                delete word from the dataset by index
                """
                if index < 0 or index >= len(self.data):
                        print("Invalid index. Please provide a valid index.")
                        return
                
                self.data.drop(index, inplace=True)
                self.data.reset_index(drop=True, inplace=True)

        def print_and_play_word_by_index(self,index):
                print(self.data['text'][index])
                ipd.display(ipd.Audio(self.data['audio'][index], rate=SR))

        def augment_audio(self, audio):
                """
                augment the audio using audiomentations.
                each augmentation is done with random values.
                """
                augment = Compose([
                    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.03, p=0.5),
                    TimeStretch(min_rate=0.8, max_rate=3, p=1),
                    PitchShift(min_semitones=-8, max_semitones=8, p=1),
                    Shift(min_shift=0, max_shift=2, shift_unit="seconds", rollover=False),
                    RoomSimulator(),
                ])
                
                return audio

In [10]:
# !pip install audiomentations[extras]

In [11]:
if NEWDATA:
    train_data = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=4, nusachim=NUSACHIM, augment=AUGMENT)

else:
    train_data = parashat_hashavua_dataset(new_data = False, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=13, prob_for_num_of_parts = [0.05, 0.05, 0.05, 0.05, 0.1, 0.15, 0.15, 0.2, 0.2])

100%|█████████████████████████████████████████| 357/357 [01:26<00:00,  4.14it/s]


success_count:  333
fail_count:  24
-25: 1
-9: 1
-8: 1
-7: 3
-5: 1
-4: 2
-2: 4
-1: 8
1: 2
2: 1
diff_list:  [(-25, 'Beshalach-4.txt'), (-9, 'Haazinu-1.txt'), (-8, 'Haazinu-2.txt'), (-7, 'Haazinu-3.txt'), (-7, 'Haazinu-6.txt'), (-7, 'Haazinu-5.txt'), (-5, 'Haazinu-4.txt'), (-4, 'Vaethanan-4.txt'), (-4, 'Yitro-6.txt'), (-2, 'Shmini-6.txt'), (-2, 'Behaalotcha-5.txt'), (-2, 'Nasso-6.txt'), (-2, 'Vayishlach-6.txt'), (-1, 'Emor-1.txt'), (-1, 'Vayeshev-6.txt'), (-1, 'Bamidbar-3.txt'), (-1, 'Devarim-5.txt'), (-1, 'LechLecha-4.txt'), (-1, 'Mishpatim-1.txt'), (-1, 'Shemot-1.txt'), (-1, 'Behar-5.txt'), (1, 'KiTisa-4.txt'), (1, 'Tetzaveh-3.txt'), (2, 'Tzav-3.txt')]


In [12]:
# train_data.check_failed_files_of_old_data()


In [13]:
# # Load cantillationless data
# train_data = parashat_hashavua_dataset(few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=4, nusachim=NUSACHIM, augment=AUGMENT, load_cantillationless_data=True)
# val_data = train_data

In [14]:
a = train_data[0]

2024-05-16 18:37:11.712365: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 18:37:12.371048: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
# play random from the train data
# train_data.print_and_play_word_by_index(random.randint(0, len(train_data.data)))


#validation of the data
# train_data.check_the_data() 
# remove the sample index:
# train_data.remove_word_by_index(32487)


In [16]:
if NEWDATA:
    val_data = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train=False ,validation=True,  num_of_words_in_sample=4, nusachim=NUSACHIM)
else:
    val_data = parashat_hashavua_dataset(new_data = False, train=False ,validation=True, num_of_words_in_sample=13, random=RANDOM)
    

print(len(train_data))
print(len(val_data))

100%|███████████████████████████████████████████| 21/21 [00:04<00:00,  4.52it/s]

success_count:  21
fail_count:  0
diff_list:  []
100000
314


In [17]:
path = "/home/user_7542/Desktop/project/cantillation/"

In [18]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [19]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [20]:
import concurrent.futures
import evaluate
import time
import cantilLocations_evaluation


# # possible metrics : "wer", "cer", "bleu", "rouge", "sacrebleu", "sari":
# # 1. `wer`: Word Error Rate.
# # 2. `cer`: Character Error Rate.
# # 3. `bleu`: Bilingual Evaluation Understudy.
# # 4. `rouge`: Recall-Oriented Understudy for Gisting Evaluation.
# # 5. `sacrebleu`: A standardized BLEU score implementation for more consistent machine translation evaluation.
# # 6. `sari`: System Agnostic Refinement Index. 

WER_CALCULATOR = evaluate.load("wer")
def compute_metrics(pred):
    eval_list = cantilLocations_evaluation.calculate_precision_recall_f1_for_string_list_with_method_list
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    # method to calculate the metrics(method can be "Exact", "Letter_Shift", "Word_Level", "Word_Shift")
    methods = ["Exact", "Letter_Shift", "Word_Level", "Word_Shift"]

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    
    start_time = time.time()
    
    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    decode_time = time.time() - start_time
    
    # evaluate the metrics
    results = eval_list(pred_str, label_str, methods)
    
    
    
    # compute the average of each metric
    avg = {}
    for i in range(4):
        avg["avg_precision_" + methods[i]] = np.mean(results[i][0])
        avg["avg_recall_" + methods[i]] = np.mean(results[i][1])
        avg["avg_f1_" + methods[i]] = np.mean(results[i][2])
    
    precision_list_exact = results[methods.index("Exact")][0]
    recall_list_exact = results[methods.index("Exact")][1]
    f1_list_exact = results[methods.index("Exact")][2]
    
    # compute the median
    precision_median_exact = np.median(precision_list_exact)
    recall_median_exact = np.median(recall_list_exact)
    f1_median_exact = np.median(f1_list_exact)
    
    
    # max and min:
    precision_max_exact = np.max(precision_list_exact)
    recall_max_exact = np.max(recall_list_exact)
    f1_max_exact = np.max(f1_list_exact)
    best_index = np.argmax(f1_list_exact)
    
    f1_min = [0, 0, 0, 0]
    recall_min = [0, 0, 0, 0]
    precision_min = [0, 0, 0, 0]
    
    for i in range(4):
        precision_min[i] = np.min(results[i][0])
        recall_min[i] = np.min(results[i][1])
        f1_min[i] = np.min(results[i][2])
    
    worst_index = [np.argmin(results[i][2]) for i in range(4)] 
    
    
    
    start_time = time.time()
    # WER
    wer = 100 * WER_CALCULATOR.compute(predictions=pred_str, references=label_str)
    
    wer_time = time.time() - start_time
    
    best_pred = pred_str[best_index]
    best_label = label_str[best_index]
    worst_pred = [pred_str[worst_index[i]] for i in range(4)]
    worst_label = [label_str[worst_index[i]] for i in range(4)]
    
    # print
    # best:
    print(f"best f1 for {methods[0]}: {f1_max_exact}\nbest pred: {best_pred}\nbest label: {best_label}\n")
    
    # worst (the worst for each method):
    for i in range(4):
        print(f"worst f1 for {methods[i]}: {f1_min[i]}\nworst pred: {worst_pred[i]}\nworst label: {worst_label[i]}\n")
    
    
    
    print("Time taken for each part:")
    print(f"Decode calculation: {decode_time} seconds")
    print(f"WER calculation: {wer_time} seconds")
    
    # matric_dict = {"wer": wer, "precision": precision_avg, "recall": recall_avg, "f1": f1_avg, "precision_median": precision_median, "recall_median": recall_median, "f1_median": f1_median, "precision_max": precision_max, "recall_max": recall_max, "f1_max": f1_max, "precision_min": precision_min, "recall_min": recall_min, "f1_min": f1_min}
    
    # create the matric_dict with the metrics
    matric_dict = {"wer": wer}
    for i in range(4):
        matric_dict["avg_precision_" + methods[i]] = avg["avg_precision_" + methods[i]]
        matric_dict["avg_recall_" + methods[i]] = avg["avg_recall_" + methods[i]]
        matric_dict["avg_f1_" + methods[i]] = avg["avg_f1_" + methods[i]]
    matric_dict["precision_median_exact"] = precision_median_exact
    matric_dict["recall_median_exact"] = recall_median_exact
    matric_dict["f1_median_exact"] = f1_median_exact
    matric_dict["precision_max_exact"] = precision_max_exact
    matric_dict["recall_max_exact"] = recall_max_exact
    matric_dict["f1_max_exact"] = f1_max_exact
    for i in range(4):
        matric_dict["precision_min_" + methods[i]] = precision_min[i]
        matric_dict["recall_min_" + methods[i]] = recall_min[i]
        matric_dict["f1_min_" + methods[i]] = f1_min[i]
    # print(matric_dict)
    return matric_dict

In [21]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL_NAME, use_cache=False) # we can add "force_download=True" to download the model again

model.generation_config.language = "he"




# # initialize the last layer of the model:
# model.proj_out.__init__(model.proj_out.in_features, len(processor.tokenizer))

# # add dropout
if DROPOUT:
    model.config.attention_dropout = DROPOUT
    model.config.dropout = DROPOUT
    model.config.activation_dropout = DROPOUT


if ADDTOKENS:
    model.resize_token_embeddings(len(processor.tokenizer))

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

model.config.decoder_input_ids = None



In [22]:
# MODEL_NAME

In [23]:
from transformers import Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir= MODEL_NAME,  # change to a repo name of your choice
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=LR, # was 1e-5
    warmup_steps=WARMUP_STEPS, # was 500
    max_steps=MAX_STEPS, # was 4000
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':False}, # I added that because UserWarning: "The default value of use_reentrant will be updated to be False in the future."
    fp16=torch.cuda.is_available(), # I added that because fp16 can't be use on CPU but on cuda
    evaluation_strategy="steps",
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=SAVE_STEPS, 
    eval_steps=EVAL_STEPS,   
    logging_steps=25, 
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model= "avg_f1_Exact",# "avg_f1_..." like "avg_f1_Exact"
    greater_is_better=True, # if we use f1 score in eval so greater is better
    push_to_hub=True,
    # I added the dataloader_prefetch_factor to support newer versions of torch (now it must be int and not None. and the default is 2).
    dataloader_prefetch_factor=2, # support newer versions of torch
    dataloader_num_workers=1, # parallelize the data loading
    weight_decay=WEIGHT_DECAY,
    run_name=RUN_NAME, # It doesn't work
)


In [24]:
from transformers import Seq2SeqTrainer, TrainerCallback


class EvaluateFirstStepCallback(TrainerCallback):
    def on_step_begin(self, args, state, control, model, tokenizer, optimizer, lr_scheduler, train_dataloader, eval_dataloader, **kwargs):
        if state.global_step == 0:
            control.should_evaluate = True



trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks = [EvaluateFirstStepCallback()] if EVALUATE_FIRST_STEP else None
)



comet_ml is installed but `COMET_API_KEY` is not set.


In [25]:
processor.save_pretrained(training_args.output_dir)

[]

In [26]:
def flags_warnings():
    if FASTTEST:
        for i in range(10):
            print("!!!TEST-MODE!!! \t \t to test the code only")

    if not ADDTOKENS:
        print("!!!ADDTOKENS==False!!!")

    if not NEWDATA:
        print("!!!NEWDATA==False!!!")

In [27]:
RUN_NAME

'medium_Random-True_OriginalData'

In [28]:
flags_warnings()

trainer_state = trainer.train()

!!!NEWDATA==False!!!


Step,Training Loss,Validation Loss,Wer,Avg Precision Exact,Avg Recall Exact,Avg F1 Exact,Avg Precision Letter Shift,Avg Recall Letter Shift,Avg F1 Letter Shift,Avg Precision Word Level,Avg Recall Word Level,Avg F1 Word Level,Avg Precision Word Shift,Avg Recall Word Shift,Avg F1 Word Shift,Precision Median Exact,Recall Median Exact,F1 Median Exact,Precision Max Exact,Recall Max Exact,F1 Max Exact,Precision Min Exact,Recall Min Exact,F1 Min Exact,Precision Min Letter Shift,Recall Min Letter Shift,F1 Min Letter Shift,Precision Min Word Level,Recall Min Word Level,F1 Min Word Level,Precision Min Word Shift,Recall Min Word Shift,F1 Min Word Shift
1,No log,2.879198,88.959660,0.156939,0.166981,0.160175,0.194556,0.207437,0.198714,0.214374,0.225630,0.217621,0.372014,0.398846,0.380862,0.108824,0.125000,0.121324,0.750000,0.769231,0.740741,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
500,0.289300,0.174671,17.983015,0.837280,0.839002,0.837340,0.861463,0.864920,0.862407,0.868347,0.871258,0.869123,0.938787,0.942428,0.939782,0.919872,0.919872,0.901613,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1000,0.209000,0.142697,14.564756,0.855607,0.862864,0.858565,0.878639,0.886241,0.881761,0.883542,0.890691,0.886477,0.951550,0.957938,0.954057,0.928571,0.928571,0.928571,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.307692,0.320000
1500,0.183100,0.138127,14.267516,0.874007,0.882451,0.877569,0.895090,0.904506,0.899139,0.898996,0.907409,0.902647,0.953600,0.960523,0.956454,0.928571,0.930952,0.933333,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.600000,0.642857,0.620690
2000,0.137100,0.137629,13.524416,0.873758,0.878869,0.875715,0.895122,0.900362,0.897123,0.898715,0.903159,0.900368,0.958810,0.962682,0.960158,0.933333,0.933333,0.933333,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.692308,0.714286,0.733333
2500,0.113800,0.135855,12.760085,0.877447,0.885933,0.881070,0.896316,0.905500,0.900271,0.900300,0.907179,0.903181,0.958160,0.966542,0.961755,0.933333,0.933333,0.960000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.600000,0.692308,0.642857
3000,0.110400,0.135647,12.845011,0.874909,0.882128,0.877963,0.891212,0.899324,0.894708,0.894007,0.901033,0.896999,0.958249,0.965729,0.961398,0.933333,0.933333,0.956522,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.600000,0.727273,0.666667
3500,0.090900,0.140882,12.420382,0.880845,0.887315,0.883507,0.900678,0.908043,0.903783,0.903124,0.910004,0.906036,0.963948,0.970578,0.966744,0.933333,0.933333,0.948849,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.600000,0.692308,0.642857
4000,0.090000,0.137013,12.016985,0.888624,0.891030,0.889326,0.905335,0.908464,0.906398,0.907921,0.910645,0.908834,0.965485,0.968453,0.966500,0.937500,0.939338,0.962963,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.733333,0.714286
4500,0.068500,0.140506,11.953291,0.891204,0.894597,0.892381,0.907875,0.912092,0.909461,0.910298,0.913987,0.911672,0.965045,0.970334,0.967188,0.941176,0.941176,0.962963,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.600000,0.642857,0.642857


best f1 for Exact: 0.7407407407407408
best pred: א֣ין פ֔ה וישכ֧יר הכה֛ן את־נ֥גע הנ֖גע שפ֥ת ימֽים׃ ורא֨ה הכה֣ין את־הנגע֮ בי֣ום השבי֒
best label: א֣ין ב֑ו והסג֧יר הכה֛ן את־ נ֥גע הנ֖תק שבע֥ת ימֽים׃ ורא֨ה הכה֣ן את־ הנ֘גע֮ בי֣ום השביעי֒

worst f1 for Exact: 0.0
worst pred: ונ֗אמר לי֨אמר֙ לכ֜ו ויומ֣י מש֔ה ל֥א נח֖וד לֽאצ֣ון כ֑ן כ֣י תֽהי֣ה ׀
worst label: מש֖ה וֽלאהר֑ן וי֗אמר לכ֛ו זבח֥ו לאלֽהיכ֖ם באֽרץ׃ וי֣אמר מש֗ה ל֤א נכון֙ לעש֣ות כ֔ן כ֚י תועב֣ת

worst f1 for Letter_Shift: 0.0
worst pred: הזאת֙ אל֣וש שיב֣ך אשא֔ה וי֤אמר יהו֣ה אל־מש֔ה וא֨ל־פרא֔ו
worst label: הז֑את ול֥א של֖ח את־ העֽם׃ וי֤אמר יהוה֙ אל־ מש֔ה ב֖א אל־ פרע֑ה ודברת֣ אל֗יו כֽה־

worst f1 for Word_Level: 0.0
worst pred: הזאת֙ אל֣וש שיב֣ך אשא֔ה וי֤אמר יהו֣ה אל־מש֔ה וא֨ל־פרא֔ו
worst label: הז֑את ול֥א של֖ח את־ העֽם׃ וי֤אמר יהוה֙ אל־ מש֔ה ב֖א אל־ פרע֑ה ודברת֣ אל֗יו כֽה־

worst f1 for Word_Shift: 0.0
worst pred: ואב֣יו יבכ֔ק וימ֥י אלה֖ים אל־נ֥ח לאמֽר׃ וי֨אמר֙ יֽעק֔ב וי֨אמר֙ אנ֣כ֣י הא֔רץ
worst label: אב֥יו יצחֽק׃ וי֨אמר אלה֤ים׀ לישראל֙ במרא֣ת 

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


this sequence (of  15  words) is too long!
sequence audio length:  30.0540625
sequence text length(in tokens):  78
text:  הש֔מש ושכ֥ב בשלמת֖ו ובֽרכ֑ך ולך֙ תהי֣ה צדק֔ה לפנ֖י יהו֥ה אלהֽיך׃ ויוס֖ף הור֣ד מצר֑ימה ויקנ֡הו פוטיפר֩
best f1 for Exact: 1.0
best pred: כב֔ד ב֥יתה פרע֖ה וב֣ית עבד֑יו ובכל־ א֧רץ מצר֛ים תשח֥ת הא֖רץ מפנ֥י הערֽב׃ ויקר֣א פרע֔ה אל־
best label: כב֔ד ב֥יתה פרע֖ה וב֣ית עבד֑יו ובכל־ א֧רץ מצר֛ים תשח֥ת הא֖רץ מפנ֥י הערֽב׃ ויקר֣א פרע֔ה אל־

worst f1 for Exact: 0.0
worst pred: עלמ֣י מריב֑ה האמ֞ר לאב֤יו ולאמו֙ ל֣א ראית֔יו ואת־ אחיו֙ ל֣א הכ֔יר ואת־ בנ֖ו ל֣א
worst label: על־ מ֥י מריבֽה׃ האמ֞ר לאב֤יו ולאמו֙ ל֣א ראית֔יו ואת־ אחיו֙ ל֣א הכ֔יר ואת־ בנ֖ו ל֣א

worst f1 for Letter_Shift: 0.0
worst pred: עלמ֣י מריב֑ה האמ֞ר לאב֤יו ולאמו֙ ל֣א ראית֔יו ואת־ אחיו֙ ל֣א הכ֔יר ואת־ בנ֖ו ל֣א
worst label: על־ מ֥י מריבֽה׃ האמ֞ר לאב֤יו ולאמו֙ ל֣א ראית֔יו ואת־ אחיו֙ ל֣א הכ֔יר ואת־ בנ֖ו ל֣א

worst f1 for Word_Level: 0.0
worst pred: עלמ֣י מריב֑ה האמ֞ר לאב֤יו ולאמו֙ ל֣א ראית֔יו ואת־ אחיו֙ ל֣א הכ֔יר ואת־ בנ֖ו

...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


this sequence (of  15  words) is too long!
sequence audio length:  49.2749375
sequence text length(in tokens):  66
text:  אר֕ור שכ֖ב עם־ צר֑עת והובא֙ הֽחיה֙ אש֣ר כנ֑ען ז֣את ליהו֖ה אלהֽיך׃ עם־ קשה־ ספ֖רה ה֥ר
best f1 for Exact: 1.0
best pred: כב֔ד ב֥יתה פרע֖ה וב֣ית עבד֑יו ובכל־ א֧רץ מצר֛ים תשח֥ת הא֖רץ מפנ֥י הערֽב׃ ויקר֣א פרע֔ה אל־
best label: כב֔ד ב֥יתה פרע֖ה וב֣ית עבד֑יו ובכל־ א֧רץ מצר֛ים תשח֥ת הא֖רץ מפנ֥י הערֽב׃ ויקר֣א פרע֔ה אל־

worst f1 for Exact: 0.0
worst pred: עלמ֥י מריבֽה׃ העמ֞ר לאב֤יו ולאמו֙ ל֣א ראית֔יו ואת־ אחיו֙ ל֣א הכ֔יר ואת־ בנ֖ו ל֣א
worst label: על־ מ֥י מריבֽה׃ האמ֞ר לאב֤יו ולאמו֙ ל֣א ראית֔יו ואת־ אחיו֙ ל֣א הכ֔יר ואת־ בנ֖ו ל֣א

worst f1 for Letter_Shift: 0.0
worst pred: עלמ֥י מריבֽה׃ העמ֞ר לאב֤יו ולאמו֙ ל֣א ראית֔יו ואת־ אחיו֙ ל֣א הכ֔יר ואת־ בנ֖ו ל֣א
worst label: על־ מ֥י מריבֽה׃ האמ֞ר לאב֤יו ולאמו֙ ל֣א ראית֔יו ואת־ אחיו֙ ל֣א הכ֔יר ואת־ בנ֖ו ל֣א

worst f1 for Word_Level: 0.0
worst pred: עלמ֥י מריבֽה׃ העמ֞ר לאב֤יו ולאמו֙ ל֣א ראית֔יו ואת־ אחיו֙ ל֣א הכ֔יר ואת־ בנ֖ו ל֣א
worst lab

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


this sequence (of  15  words) is too long!
sequence audio length:  49.4829375
sequence text length(in tokens):  74
text:  את־ אוצר֨ו כל־ זכ֔ר תחת֔יו שא֖ול לפנ֣י יהו֑ה חטא֥ה גדלֽה׃ קשה־ ע֖רף הֽצפרדע֖ים ת֣חת בעדת־
this sequence (of  15  words) is too long!
sequence audio length:  31.53975
sequence text length(in tokens):  80
text:  על֛יו הכה֖ן וטהֽר׃ לכֽדנו׃ מֽערע֡ר אשר֩ המלכ֔ים אש֥ר מלכ֖ו אכ֣ל מחמ֗צת ונכרת֞ה ושלשֽים׃ א֣לה פקוד֗י
this sequence (of  15  words) is too long!
sequence audio length:  53.7701875
sequence text length(in tokens):  69
text:  טהֽור׃ וילכ֖ו אח֑יו את־ קל֖ו ברית֖י אתכ֑ם קשה־ ע֖רף ובמקנ֥ה הצ֛אן משחת֤ם בהם֙ משז֖ר תעש֥ה
best f1 for Exact: 1.0
best pred: כב֔ד ב֥יתה פרע֖ה וב֣ית עבד֑יו ובכל־ א֧רץ מצר֛ים תשח֥ת הא֖רץ מפנ֥י הערֽב׃ ויקר֣א פרע֔ה אל־
best label: כב֔ד ב֥יתה פרע֖ה וב֣ית עבד֑יו ובכל־ א֧רץ מצר֛ים תשח֥ת הא֖רץ מפנ֥י הערֽב׃ ויקר֣א פרע֔ה אל־

worst f1 for Exact: 0.0
worst pred: בנ֨י ישרא֜ל את־ יעק֣ב אביה֗ם ואת־ טפם֙ ואת־ נשיה֔ם בעגל֕ות אשר־ של֥ח פרע֖ה לש֥את אתֽו׃ ויקח֣ו

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}
There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


In [29]:
kwargs = {
    # "dataset_args": "config: he, split: test",
    "language": "he",
    "model_name": "he-cantillation",
    "finetuned_from": BASE_MODEL_NAME,
    "tasks": "automatic-speech-recognition-cantillation",
    "tags": "hf-asr-leaderboard",
}

In [30]:
trainer.save_model()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


In [31]:
trainer.push_to_hub(**kwargs)
# processor.push_to_hub("cantillation" +training_args.output_dir[1:])

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


CommitInfo(commit_url='https://huggingface.co/cantillation/Teamim-medium_Random-True_OriginalData/commit/bc4ad6d5f66e0fc830c7d19a6ba21a4eb62fa73b', commit_message='End of training', commit_description='', oid='bc4ad6d5f66e0fc830c7d19a6ba21a4eb62fa73b', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
trainer.lr_scheduler.get_lr()

[4.081632653061225e-09, 4.081632653061225e-09]

In [33]:
processor.tokenizer.decode(train_data[26]["labels"])

'<|startoftranscript|><|he|><|transcribe|><|notimestamps|>לחטֽאת׃ ולז֣בח השלמים֮ שש֥ה עש֖ר אדנ֑ים א֖יש אשר־ נשא֣ו מש֔ה ב֖א בק֗ש להדֽיחך֙ ע֥בד לֽמו׃<|endoftext|>'

In [34]:

from datetime import datetime


def log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="training_log.md"):
    # Get the current date and time
    now = datetime.now()

    # Format the date and time as a string
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

    with open(filename, 'a') as f:
        f.write(f"| {date_time} | {training_args.output_dir } | {training_args.per_device_train_batch_size} | {training_args.gradient_accumulation_steps} | {training_args.learning_rate} | {training_args.warmup_steps} | {training_args.max_steps} | {training_args.gradient_checkpointing} | {training_args.gradient_checkpointing_kwargs} | {training_args.fp16} | {training_args.evaluation_strategy} | {training_args.per_device_eval_batch_size} | {training_args.predict_with_generate} | {training_args.generation_max_length} | {training_args.save_steps} | {training_args.eval_steps} | {training_args.logging_steps} | {training_args.report_to} | {training_args.load_best_model_at_end} | {training_args.metric_for_best_model} | {training_args.greater_is_better} | {training_args.push_to_hub} | {training_loss} | {epoch} | {step} | {validation_loss} | {f1} | {recall} | {precision} |\n")

def create_markdown_file_with_headers(filename="training_log_new.md"):
    with open(filename, 'w') as f:
        f.write("| Date Time | Repo Name | Batch Size | Gradient Accumulation Steps | Learning Rate | Warmup Steps | Max Steps | Gradient Checkpointing | Gradient Checkpointing Kwargs | FP16 | Evaluation Strategy | Eval Batch Size | Predict with Generate | Max Length | Save Steps | Eval Steps | Logging Steps | Report To | Load Best Model at End | Metric for Best Model | Greater is Better | Push to Hub | Training Loss | Epoch | Step | Validation Loss | f1 | recall | precision |\n")
        f.write("|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|----|---|---|\n")

# Create the Markdown file with headers
#create_markdown_file_with_headers()
        

def get_logs_with_step(trainer, step = 1500):
    # Initialize an empty dictionary to store the merged logs
    merged_logs_with_step = {}

    # Iterate over the log history
    for log in trainer.state.log_history:
        # Check if the 'step' key exists in the log and if it equals the provided step
        if 'step' in log and log['step'] == step:
            # If it does, merge the log into the merged_logs_with_step dictionary
            merged_logs_with_step.update(log)

    # Return the merged logs
    return merged_logs_with_step


# Get the training loss
training_loss = trainer_state.training_loss
# Get the step and epoch from the TrainerState
step = trainer.state.global_step
epoch = trainer.state.epoch

# Get the log history at the specified step
history = get_logs_with_step(trainer,training_args.max_steps)
# Get the evaluation details from the log history
validation_loss = history['eval_loss']
f1 = history['eval_avg_f1_Exact']
recall = history['eval_avg_recall_Exact']
precision = history['eval_avg_precision_Exact']

# Log the training details
log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="training_log_new.md")


In [35]:
#load the markdown file
from IPython.display import Markdown

# Open the file in read mode
with open('training_log_new.md', 'r') as file:
    # Read the content of the file
    content = file.read()

# Display the content as Markdown
display(Markdown(content))

| Date Time | Repo Name | Batch Size | Gradient Accumulation Steps | Learning Rate | Warmup Steps | Max Steps | Gradient Checkpointing | Gradient Checkpointing Kwargs | FP16 | Evaluation Strategy | Eval Batch Size | Predict with Generate | Max Length | Save Steps | Eval Steps | Logging Steps | Report To | Load Best Model at End | Metric for Best Model | Greater is Better | Push to Hub | Training Loss | Epoch | Step | Validation Loss | f1 | recall | precision |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|----|---|---|
| 03/21/2024, 08:40:08 | ./Teamim-AllNusah-whisper-small_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.021550073864677687 | 8.0 | 100000 | 0.184482142329216 | 0.8996304230096901 | 0.8996085856138029 | 0.9003176340674156 |
| 03/21/2024, 13:03:45 | ./Teamim-AllNusah-whisper-base_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.03545441636119969 | 8.0 | 100000 | 0.20118673145771027 | 0.8617680742203201 | 0.8626021186921445 | 0.8617811978432426 |
| 03/21/2024, 16:41:01 | ./Teamim-AllNusah-whisper-tiny_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.04949023559953086 | 8.0 | 100000 | 0.21391823887825012 | 0.8324883831395741 | 0.8338224092406237 | 0.8322131178574591 |
| 03/23/2024, 07:00:56 | ./Teamim-AllNusah-whisper-medium_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.016411178880098305 | 8.0 | 100000 | 0.19175758957862854 | 0.9241458540823655 | 0.9248030309610401 | 0.9241210924324863 |
| 05/15/2024, 04:43:26 | ./Teamim-medium_Random-True_OriginalData | 8 | 1 | 1e-05 | 100 | 20000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 200 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.06837314007226378 | 1.6 | 20000 | 0.32565802335739136 | 0.7752270688239551 | 0.7795095778021163 | 0.7720285537597026 |
| 05/16/2024, 20:40:49 | ./Teamim-medium_Random-True_OriginalData | 8 | 1 | 1e-05 | 100 | 5000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 1000 | 500 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.1853820906639099 | 0.4 | 5000 | 0.14098824560642242 | 0.892768311387271 | 0.8960657159022651 | 0.890549476438386 |


| Model Name | Model Name | data | steps | lr |
|----------|----------|----------|--------|--------|
| whisper-medium-he-teamim-base | medium | all | 10,000 | 3e-5 |
| whisper-medium-he-teamim-ashkenazi-01 | base | ashkenazi | 9,000 | 1e-6 |




In [ ]:
# run a web server to see the tensorboard
!tensorboard --logdir ./whisper-medium-he-teamim-aviv-base --port 6006 --bind_all

2024-05-16 20:40:50.477979: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 20:40:51.051523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-16 20:40:51.805602: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-16 20:40:51.834281: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Fo

# Test the model

In [ ]:
# load the model
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
processor = WhisperProcessor.from_pretrained(MODEL_NAME)

# load the test data
test_data = parashat_hashavua_dataset(few_data=FASTTEST, train=False ,validation=False, test=True,  num_of_words_in_sample=4, nusachim=NUSACHIM, ra)

# create the data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [ ]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    eval_dataset=test_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# evaluate the model
results = trainer.evaluate() # we use evaluate to get the metrics
print(results)
# save the results to a json file
# create the results file
with open(f"results_{MODEL_NAME.split('/')[-1]}.json", 'w') as f:
    json.dump(results, f, indent=4)


In [ ]:
{'eval_loss': 0.7434155344963074, 'eval_wer': 12.154756685664182, 'eval_avg_precision_Exact': 0.9045080330977024, 'eval_avg_recall_Exact': 0.9046995984902677, 'eval_avg_f1_Exact': 0.9041808511811215, 'eval_avg_precision_Letter_Shift': 0.9262739837538204, 'eval_avg_recall_Letter_Shift': 0.9265417247288052, 'eval_avg_f1_Letter_Shift': 0.9259759561670436, 'eval_avg_precision_Word_Level': 0.9287635843313802, 'eval_avg_recall_Word_Level': 0.9291329677264785, 'eval_avg_f1_Word_Level': 0.928526987815817, 'eval_avg_precision_Word_Shift': 0.9744189603225837, 'eval_avg_recall_Word_Shift': 0.9756642194342305, 'eval_avg_f1_Word_Shift': 0.9746023104771994, 'eval_precision_median_exact': 1.0, 'eval_recall_median_exact': 1.0, 'eval_f1_median_exact': 1.0, 'eval_precision_max_exact': 1.0, 'eval_recall_max_exact': 1.0, 'eval_f1_max_exact': 1.0, 'eval_precision_min_Exact': 0.0, 'eval_recall_min_Exact': 0.0, 'eval_f1_min_Exact': 0.0, 'eval_precision_min_Letter_Shift': 0.0, 'eval_recall_min_Letter_Shift': 0.0, 'eval_f1_min_Letter_Shift': 0.0, 'eval_precision_min_Word_Level': 0.0, 'eval_recall_min_Word_Level': 0.0, 'eval_f1_min_Word_Level': 0.0, 'eval_precision_min_Word_Shift': 0.0, 'eval_recall_min_Word_Shift': 0.0, 'eval_f1_min_Word_Shift': 0.0, 'eval_runtime': 1534.2005, 'eval_samples_per_second': 1.755, 'eval_steps_per_second': 0.055}